# **Sumário** 📘


1.   **Resumo**
2.   **Introdução**
3.   **Objetivos**
4.   **Metodologia**
4.   **Modelo**
4.   **Usando o Modelo**
5.   **Otimizando o Modelo**
5.   **Visualisando parametros otimizados**
5.   **Resultados**
6.   **Referências**



## **Ferramenta de Benchmarking para Transistores de Película Fina** ⚡

## **Aluno**:

Rodrigo Santos Batista – rsb6@cin.ufpe.br

CIn ­ UFPE

## **Projeto de Pesquisa**:

Avaliação comparativa de tecnologias emergentes para computação próxima e

dentro de sensores (BEeTle)

## **Coordenador**:

Stefan Michael Blawid – sblawid@cin.ufpe.br

CIn ­ UFPE


### **Resumo** 📩

> Transistores de película fina são chaves eletrônicas com uma corrente fluindo do terminal fonte para o terminal dreno controlada pela tensão aplicada ao terceiro terminal, a porta. Os modelos de benchmark precisam reproduzir razoavelmente bem as características de corrente-tensão (IV) dos TFTs com um número mínimo de parâmetros. Aqui, razoavelmente significa que os parâmetros do modelo podem ser extraídos de forma confiável e sem ambiguidade das curvas IV experimentais.





## **Usando o Modelo** 💻


### **Carregando dados da amostra** 📚


---



In [1]:
# Load the used classes
from modules_otft._imports import *
from modules_otft._optmization import ModelOptimization
from modules_otft._read_data import ReadData 
from modules_otft._model import TFTModel 
from modules_otft._grafics import  TFTGraphicsPlot
from modules_otft._menu import TFTMenu 

from modules_otft._global_vars import * 
from modules_otft._config import *


In [2]:
# Instantiating the used classes
plot = TFTGraphicsPlot()
menu = TFTMenu()
read = ReadData()

In [3]:
show_varibles()

| path: datas/Fig1ab/tipo p
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_transfer: A
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_output: uA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| current_typic: uA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| loaded_voltages: -2, -50, -30, -40, -50
-------------------------------

In [4]:
try:
    if isinstance(eval(loaded_voltages), int):
        ld_voltages = int(loaded_voltages)
    elif len(loaded_voltages) > 2:
        ld_voltages = list(eval(loaded_voltages))
except SyntaxError:
    print("No parameters entered in 'loaded_voltages', please load them\n")


# Do not delete these lines
scale_trfr = experimental_data_scale_transfer
scale_out = experimental_data_scale_output
curr_typic = current_typic
list_tension = ld_voltages
file_path = path

if type_curve_plot == 'logarithmic':
    type_curve_plot = 'log'

max_value = len(ld_voltages) - curves_transfer
shift_list = []
try:
    if shift_volt_data =="":
        shift_list = [x*0 for x in range(max_value)]

    # Checks if the passed value is an integer
    elif isinstance(eval(shift_volt_data), int):
        shift_list.append(int(shift_volt_data))

    # Insert the passed values into a list as a list
    elif len(shift_volt_data) > 1:
        shift_list = list(eval(shift_volt_data))

except ValueError:
    print("No shift value passed, please enter a value\n")


# Returns the shifted list with the passed voltage value [V]
list_tension_shift = read.apply_shifts(curves_transfer, shift_list, ld_voltages)

path_voltages = read.read_files_experimental(file_path, list_tension_shift)

Vv, Id, input_voltage, n_points, count_transfer, count_output = read.load_data(type_read_data_exp, path_voltages, curr_typic, scale_trfr, scale_out, type_curve_plot)

plot.set_count_transfer(count_transfer)

print("Shift value    : " + f'{shift_list}')
print()

print("List volt shift: " +  f'{list_tension_shift}')
print()

list_curves = menu.view_path_reads(path_voltages, list_tension_shift)

print()
print('The table above shows all the files that were read in the directory. If you want to filter files so that only those')
print('of interest to you are read, do this by passing numbers corresponding to the order of the experiment curves,')
print('separated by commas. For example: 0:transfer-5v, 3:output-40v, and so on.')
print('\n\n')

f_selection = []
if select_files == "":
    # Call the function to read and sort the .csv files and obtain the additional lists
    path_voltages = read.read_files_experimental(file_path, list_tension_shift)
else:
    try:
        # Checks if the passed value is an integer
        if isinstance(eval(select_files), int):
            f_selection.append(int(select_files))

        # Insert the passed values into a list as a list
        elif len(select_files) > 1:
            f_selection = list(eval(select_files))

        else:
            print("No value available\n")

    except NameError:
        print("No value available, please enter a valid value\n")


    new_files_filter, new_values_tension, new_list_tension = read.filter_files(f_selection, list_curves, list_tension_shift, ld_voltages)
    path_voltages = read.read_files_experimental(file_path, new_values_tension, selected_files=new_files_filter)

    Vv, Id, input_voltage, n_points, count_transfer, count_output = read.load_data(type_read_data_exp, path_voltages, curr_typic, scale_trfr, scale_out, type_curve_plot)
    list_tension_shift = new_values_tension
    list_tension = new_list_tension

print()

print("List volt shift  : " + f'{list_tension_shift}')
print("List tension     : " + f'{list_tension}')
print()

_ = menu.view_path_reads(path_voltages, list_tension_shift)
print('\n\n')

# Type of transistor (1: npn) or (-1: pnp).
if type_of_transistor == 'nFET':
    tp_tst = 1 # n-type
else:
    tp_tst = -1 # p-type

# Resistance
if resistance_scale == "":
    print("No value of 'RESISTANCE' loaded\n")
else:
    resistance = float(resistance_scale)

# Current
if current_carry == "":
    print("No value of 'CURRENT' loaded\n")
else:
    current = float(current_carry)
    menu.set_curr_carry(current)
  
# Transistor width
if with_transistor == "":
    width_t = float(0.1000)
else:
    width_t = float(with_transistor)

print()

load_parameters = []
try:
    load_parameters = list(loaded_parameters.values())
    load_coefficient = load_parameters
except SyntaxError:
    print("No parameters entered, please load them\n")

print()

menu.print_values(load_coefficient)
print()

mode_idleak = 0
load_idleak = []

if idleak_mode == 'unique idleak value':
    mode_idleak = 0
    try:
        load_idleak = float(loaded_idleak)
    except SyntaxError:
        print("No parameters entered, please load them\n")

elif idleak_mode == 'more than one value idleak':
    mode_idleak = 1
    try:
        load_idleak = list(eval(loaded_idleak))
    except SyntaxError:
        print("No parameters entered, please load them\n")
print('\n\n')


Shift value    : [0, 0, 0]

List volt shift: [-2, -50, -30, -40, -50]

--------------------------------------------
--------------------------------------------
|                READ FILES                |
--------------------------------------------
--------------------------------------------
|            transfer-2V.csv            |
--------------------------------------------
|             transfer-50V.csv             |
--------------------------------------------
|              output-30V.csv              |
--------------------------------------------
|              output-40V.csv              |
--------------------------------------------
|              output-50V.csv              |
--------------------------------------------
--------------------------------------------

The table above shows all the files that were read in the directory. If you want to filter files so that only those
of interest to you are read, do this by passing numbers corresponding to the order of the exper

In [5]:
if (image_size_width or image_size_height) == None:
  image_size_width = 1100 #
  image_size_height = 600 #

# Instantiate the graphics class to visualize the model
plot = TFTGraphicsPlot(image_size_height, image_size_width)


# Create an instance of the model passing all the data obtained above
model = read.create_models_datas(TFTModel, n_points, type_curve_plot, load_parameters, input_voltage, Vv,
                                 load_idleak, width_t, count_transfer, tp_tst=tp_tst, scale_factor=scale_trfr,
                                 current_typic=curr_typic, res=resistance, curr=current)

# Reads the experimental and model (transfer) data to be displayed on the graph
in_model_data, in_exp_data = read.group_by_trasfer(count_transfer, Vv, Id, model)

# Reads the experimental and model (output) data to be displayed on the graph
out_model_data, out_exp_data = read.group_by_output(count_output, count_transfer, Vv, Id, model)


option = 'show both curves' # @param ["Show Transfer curve", "Show output curve", "show both curves"]
if option == 'Show Transfer curve':
  clear_output(wait=True) 
  plot.plot_vgs_vds(list_tension, list_tension_shift, 0, count_transfer, in_model_data, shift_list, select_files, *in_exp_data, sample_unit=current_typic, plot_type=type_curve_plot)

elif option == 'Show output curve':
  clear_output(wait=True) 
  plot.plot_vgs_vds(list_tension, list_tension_shift, 1, count_transfer, out_model_data, shift_list, select_files, *out_exp_data, sample_unit=current_typic, plot_type='linear')


elif option == 'show both curves':
  clear_output(wait=True) 
  plot.plot_vgs_vds(list_tension, list_tension_shift, 0, count_transfer, in_model_data, shift_list, select_files, *in_exp_data, sample_unit=current_typic, plot_type=type_curve_plot)
  print()
  plot.plot_vgs_vds(list_tension, list_tension_shift, 1, count_transfer, out_model_data, shift_list, select_files, *out_exp_data, sample_unit=current_typic, plot_type='linear')

else:
  print("No option choice\n")

print()
read.compute_relative_distance(Id, model)


0
1



0
1
2



**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 147082113134.7026

**************************************************************************************************************************************************


### **Model Optimization** ⚡

This optimization section focuses on estimating the optimal parameters for a MOSFET transistor given a sequence of input voltage $V$ [V] and current $I$ [mA], using the mathematical model introduced in section [5]. The goal here is to estimate these optimal parameters using the `curve_fit` function, as also introduced in section [4] (for more details, refer to that section). The parameters in question are listed below:

- $V_{tho}$: Gate threshold voltage of the transistor.
- $\Delta$: Channel modulation coefficient.
- $N$: Number of transistor channels.
- $L$: Channel length of the transistor.
- $\Lambda$: Channel modulation length.
- $V_{crit}$: Transistor breakdown voltage.
- $J_{th}$: Transistor saturation current.
- $R_s$: Transistor series resistance.



In [6]:
# Lower Bounds
lw_bounds = list(lower_bounds.values())

# Upper Bounds
up_bounds = list(upper_bounds.values())

# Tolerance factor for optimization
if tolerance_factor == "":
    print("No tolerance factor loaded\n")
    tlr_factor = float(4e-6)
else:
    tlr_factor = float(tolerance_factor)

# Reset bounds if default_bounds is set to 'yes'
if default_bounds == 'yes':
    lw_bounds = None
    up_bounds = None

# Creation of the model that the optimizer will use to estimate the best parameters
model_id = TFTModel(input_voltage, n_points, type_curve_plot, current_typic=curr_typic,
                    scale_factor=scale_trfr, idleak=load_idleak, mult_idleak=mode_idleak,
                    curv_transfer=count_transfer, sr_resistance=resistance, curr_carry=current)

# Create an instance of the TFTOptimization class
optimizer = ModelOptimization(current_typic=curr_typic, scale_transfer=scale_trfr, scale_output=scale_out,
                              type_read=type_read_data_exp, path_voltages=path_voltages, type_curve=type_curve_plot, method=optimization_method,
                              bounds=(lw_bounds, up_bounds))

# Set the default bounds
optimizer.set_default_bounds(default_bounds)
# Set the convergence tolerance
optimizer.set_ftol_param(tlr_factor)

# Model Optimization
coeff_opt, coeff_error, text_verbose = optimizer.optimize_all(model_id, load_coefficient, *path_voltages)

print('\n\n\n')
print('*'*200)
print(text_verbose.capitalize())
print('*'*200)
print('\n\n\n')



----------------------------------------------------------------------------------------------------
TRUST REGION REFLECTIVE (TRF) MODE
----------------------------------------------------------------------------------------------------





********************************************************************************************************************************************************************************************************
`gtol` termination condition is satisfied.
function evaluations 89, initial cost 2.3642e+05, final cost 5.0757e-01, first-order optimality 3.94e-07.

********************************************************************************************************************************************************************************************************






In [7]:
option = 'Show the table of values opt' # @param ["Show the coeff values","Show the initial values", "Show the table of values opt", "coeff opt", "coeff error"]

if option == 'Show the coeff values':
  menu.print_values(load_coefficient)

elif option == 'Show the initial values':
  menu.print_values(load_coefficient)

elif option == 'coeff opt':
  menu.print_values(coeff_opt)

elif option == 'coeff error':
  menu.print_values(coeff_error)

elif option == 'Show the table of values opt':
  menu.show_table_info(load_coefficient, load_coefficient, coeff_opt, coeff_error, curr_typic, resistance)

print('\n\n\n')



---------------------------
 OPTIMIZED COEFFICIENTS:

╒══════════════════════╤═════════════════╤═══════════════════╤═════════════════════════════════════════╕
│ Parameter            │   Initial Value │   Optimized Value │   Associated Error (standard deviation) │
╞══════════════════════╪═════════════════╪═══════════════════╪═════════════════════════════════════════╡
│ VTHO [V]             │     8.57        │      12           │                              9.75017    │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤
│ DELTA                │     0.0123      │       0.00714077  │                              0.0322358  │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤
│ N                    │   108           │     120.986       │                             51.9999     │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤


### **Comparing** ✅

---
This section aims to evaluate the model's performance relative to the initial data. Visual observation allows assessing whether the model significantly outperformed the initial coefficients.

In this section, simply execute the cells below; no explicit modifications are required.

---

In [8]:
# Create new models with optimized coefficients to compare with the initial ones
model_opt = read.create_models_datas(TFTModel, n_points, type_curve_plot, coeff_opt,
                                     input_voltage, Vv, load_idleak, width_t, count_transfer,
                                     tp_tst=tp_tst, scale_factor=scale_trfr, current_typic=curr_typic,
                                     res=resistance, curr=current)

# Reads the experimental and model (transfer) data to be displayed on the graph
in_model_data_opt, in_exp_data_opt = read.group_by_trasfer(count_transfer, Vv, Id, model_opt)


# Reads the experimental and model (output) data to be displayed on the graph
out_model_data_opt, out_exp_data_opt = read.group_by_output(count_output,count_transfer, Vv, Id, model_opt)


# Inserting the model and experimental curves to visualize if there is significant discrepancy in the optimization
in_model_data_comp, in_exp_data_comp =  read.group_by_trasfer(count_transfer, Vv, Id, model, model_opt, compare=True)


# Inserting the model and experimental curves to visualize if there is significant discrepancy in the optimization.
out_model_data_comp, out_exp_data_comp = read.group_by_output(count_output, count_transfer, Vv, Id, model, model_opt, compare=True)


option = 'show both curves comp' # @param ["Show Transfer curve opt", "Show output curve opt", "show both curves opt", "Show Transfer curve comp", "Show output curve comp", "show both curves comp"]

if option == 'Show Transfer curve opt':
  plot.plot_vgs_vds(list_tension, list_tension_shift, 0, count_transfer,
                    in_model_data_opt, shift_list, select_files, *in_exp_data_opt, sample_unit=curr_typic, plot_type=type_curve_plot)

elif option == 'Show output curve opt':
  plot.plot_vgs_vds(list_tension, list_tension_shift, 1, count_transfer,
                    out_model_data_opt, shift_list, select_files, *out_exp_data_opt, sample_unit=curr_typic)

elif option == 'show both curves opt':
  plot.plot_vgs_vds(list_tension, list_tension_shift, 0, count_transfer,
                    in_model_data_opt, shift_list, select_files, *in_exp_data_opt, sample_unit=curr_typic, plot_type=type_curve_plot)
  print()
  plot.plot_vgs_vds(list_tension, list_tension_shift, 1, count_transfer,
                    out_model_data_opt, shift_list, select_files, *out_exp_data_opt, sample_unit=curr_typic)

elif option == 'Show Transfer curve comp':
  plot.plot_vgs_vds(list_tension, list_tension_shift, 0, count_transfer, in_model_data_comp, shift_list, select_files,
                    *in_exp_data_comp, sample_unit=curr_typic, plot_type=type_curve_plot, compare=True)

elif option == 'Show output curve comp':
  plot.plot_vgs_vds(list_tension, list_tension_shift, 1, count_transfer, out_model_data_comp, shift_list, select_files,
                    *out_exp_data_comp, sample_unit=curr_typic, compare=True)


elif option == 'show both curves comp':
  plot.plot_vgs_vds(list_tension, list_tension_shift, 0, count_transfer, in_model_data_comp, shift_list, select_files,
                    *in_exp_data_comp, sample_unit=curr_typic, plot_type=type_curve_plot, compare=True)
  print()
  plot.plot_vgs_vds(list_tension, list_tension_shift, 1, count_transfer, out_model_data_comp, shift_list, select_files,
                    *out_exp_data_comp, sample_unit=curr_typic, compare=True)

else:
  print("No option choice\n")

print()
read.compute_relative_distance(Id, model_opt)
print('\n\n\n\n\n')


0
1
2
3



0
1
2
3
4
5



**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 9738880877.4632

**************************************************************************************************************************************************






